In [9]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

# Load the MovieLens dataset (You can download it from https://grouplens.org/datasets/movielens/)
movies = pd.read_csv('movies.csv')
ratings = pd.read_csv('ratings.csv')

# Merge the datasets to get user-item ratings
movie_ratings = pd.merge(ratings, movies, on='movieId')

# Create a user-item matrix
user_movie_ratings = pd.pivot_table(movie_ratings, index='userId', columns='title', values='rating')

# Fill missing values with 0
user_movie_ratings = user_movie_ratings.fillna(0)

# Calculate similarity between movies using cosine similarity
movie_similarity = cosine_similarity(user_movie_ratings.T)

# Create a DataFrame for movie recommendations
movie_similarity_df = pd.DataFrame(movie_similarity, index=user_movie_ratings.columns, columns=user_movie_ratings.columns)

# Define a function to get movie recommendations for a given movie
def get_movie_recommendations(movie_title, num_recommendations=5):
    similar_movies = movie_similarity_df[movie_title].sort_values(ascending=False)
    similar_movies = similar_movies.drop(movie_title)
    recommendations = similar_movies.head(num_recommendations)
    return recommendations.index.tolist()

# Get movie recommendations for a specific movie
movie_title = 'Toy Story (1995)'
recommendations = get_movie_recommendations(movie_title)

# Print the recommendations
print(f"Recommended movies similar to '{movie_title}':")
for movie in recommendations:
    print(movie)

Recommended movies similar to 'Toy Story (1995)':
Toy Story 2 (1999)
Jurassic Park (1993)
Independence Day (a.k.a. ID4) (1996)
Star Wars: Episode IV - A New Hope (1977)
Forrest Gump (1994)


In [11]:
import pandas as pd
from sklearn.neighbors import NearestNeighbors

# Load the MovieLens dataset 
movies = pd.read_csv('movies.csv')
ratings = pd.read_csv('ratings.csv')

# Create a user-item matrix
user_movie_ratings = pd.pivot_table(ratings, index='userId', columns='movieId', values='rating', fill_value=0)

# Instantiate and fit a Nearest Neighbors model
knn = NearestNeighbors(n_neighbors=5, metric='cosine', algorithm='brute')
knn.fit(user_movie_ratings.values.T)

# Function to get movie recommendations for a given movie
def get_movie_recommendations(movie_title, num_recommendations=5):
    movie_idx = movies[movies['title'] == movie_title].index[0]
    _, indices = knn.kneighbors(user_movie_ratings.iloc[:, movie_idx].values.reshape(1, -1), n_neighbors=num_recommendations+1)
    recommended_movie_indices = indices[0][1:]  # Exclude the input movie itself
    recommendations = movies.iloc[recommended_movie_indices]['title'].tolist()
    return recommendations

# Get movie recommendations for a specific movie
movie_title = 'Toy Story (1995)'
recommendations = get_movie_recommendations(movie_title)

# Print the recommendations
print(f"Recommended movies similar to '{movie_title}':")
for i, movie in enumerate(recommendations, start=1):
    print(f"{i}. {movie}")


Recommended movies similar to 'Toy Story (1995)':
1. 'night Mother (1986)
2. Jurassic Park (1993)
3. Independence Day (a.k.a. ID4) (1996)
4. Star Wars: Episode IV - A New Hope (1977)
5. Forrest Gump (1994)
